# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
import re
from sklearn.metrics import classification_report
from sklearn.base import BaseEstimator, TransformerMixin
import pickle

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [2]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('InsertTableName', engine)
X = df['message']
Y = df.drop(['id', 'message', 'original', 'genre'], axis=1)

### 2. Write a tokenization function to process your text data

In [4]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

def tokenize(text):
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
pipeline = Pipeline([ 
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier())) ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=1)
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
y_pred = pipeline.predict(X_test)

In [8]:
columns = y_test.columns
for i in range(len(columns)):
    print("Category", i, columns[i])
    print(classification_report(y_test.iloc[:,i], y_pred[:,i]))

Category 0 related
             precision    recall  f1-score   support

          0       0.62      0.35      0.45      2047
          1       0.82      0.93      0.87      6604

avg / total       0.77      0.79      0.77      8651

Category 1 request
             precision    recall  f1-score   support

          0       0.88      0.98      0.93      7137
          1       0.83      0.38      0.52      1514

avg / total       0.87      0.88      0.86      8651

Category 2 offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      8616
          1       0.00      0.00      0.00        35

avg / total       0.99      1.00      0.99      8651

Category 3 aid_related
             precision    recall  f1-score   support

          0       0.72      0.88      0.79      5059
          1       0.75      0.51      0.60      3592

avg / total       0.73      0.73      0.71      8651

Category 4 medical_help
             precision    recall  f1-s

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [9]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7fbdd9f9b9d8>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

In [16]:
parameters = {'clf__estimator__n_estimators': [10, 20],
              'clf__estimator__min_samples_split': [2, 4]}

cv = GridSearchCV(pipeline, param_grid=parameters, n_jobs=-1, verbose=3)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [17]:
cv.fit(X_train, y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, score=0.2170794192997438, total=  37.9s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10 


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:   46.3s remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, score=0.21895815542271563, total=  38.1s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10 


[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:  1.5min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, score=0.21984967543559958, total=  37.5s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=20 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=20, score=0.2363791631084543, total= 1.1min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=20 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=20, score=0.23347566182749788, total= 1.1min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=20 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=20, score=0.2268534335497096, total= 1.1min
[CV] clf__estimator__min_samples_split=4, clf__estimator__n_estimators=10 
[CV]  clf__estimator__min_samples_split=4, clf__estimator__n_estimators=10, score=0.2174210076857387, total=  33.3s
[CV] clf__estimator__min_samples_split=4, clf__estimator__n_estimators=10 
[CV]  clf__estimator__min_samples_split=4, 

[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed: 11.6min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'clf__estimator__n_estimators': [10, 20], 'clf__estimator__min_samples_split': [2, 4]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=3)

In [18]:
print(cv.best_params_)
print(cv.best_score_)

{'clf__estimator__min_samples_split': 2, 'clf__estimator__n_estimators': 20}
0.232236392621


In [19]:
y_pred = cv.predict(X_test)

In [20]:
for i in range(len(columns)):
    print("Category", i, columns[i])
    print(classification_report(y_test.iloc[:,i], y_pred[:,i]))

Category 0 related
             precision    recall  f1-score   support

          0       0.71      0.31      0.43      2047
          1       0.82      0.96      0.88      6604

avg / total       0.79      0.81      0.78      8651

Category 1 request
             precision    recall  f1-score   support

          0       0.88      0.99      0.93      7137
          1       0.89      0.39      0.54      1514

avg / total       0.89      0.88      0.87      8651

Category 2 offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      8616
          1       0.00      0.00      0.00        35

avg / total       0.99      1.00      0.99      8651

Category 3 aid_related
             precision    recall  f1-score   support

          0       0.74      0.89      0.81      5059
          1       0.78      0.55      0.65      3592

avg / total       0.75      0.75      0.74      8651

Category 4 medical_help
             precision    recall  f1-s

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [21]:
# Using the StartingVerbExtractor from course exercises

class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

**First attempt: Adding StartingVerbExtractor**

In [22]:
pipeline_2 = Pipeline([ 
    ('features', FeatureUnion([

        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),

        ('starting_verb', StartingVerbExtractor())
    ])),

#    ('clf', MultiOutputClassifier(AdaBoostClassifier()))
     ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=1)
pipeline_2.fit(X_train, y_train)
y_pred = pipeline_2.predict(X_test)

In [24]:
for i in range(len(columns)):
    print("Category", i, columns[i])
    print(classification_report(y_test.iloc[:,i], y_pred[:,i]))

Category 0 related
             precision    recall  f1-score   support

          0       0.63      0.35      0.45      2047
          1       0.82      0.94      0.88      6604

avg / total       0.78      0.80      0.77      8651

Category 1 request
             precision    recall  f1-score   support

          0       0.88      0.98      0.93      7137
          1       0.83      0.36      0.50      1514

avg / total       0.87      0.88      0.85      8651

Category 2 offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      8616
          1       0.00      0.00      0.00        35

avg / total       0.99      1.00      0.99      8651

Category 3 aid_related
             precision    recall  f1-score   support

          0       0.71      0.88      0.78      5059
          1       0.74      0.49      0.59      3592

avg / total       0.72      0.72      0.70      8651

Category 4 medical_help
             precision    recall  f1-s

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


**Second attempt: Using AdaBoostClassifier**

In [25]:
pipeline_3 = Pipeline([ 
    ('features', FeatureUnion([

        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),

        ('starting_verb', StartingVerbExtractor())
    ])),

       ('clf', MultiOutputClassifier(AdaBoostClassifier()))
])

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=1)
pipeline_3.fit(X_train, y_train)
y_pred = pipeline_3.predict(X_test)

In [27]:
for i in range(len(columns)):
    print("Category", i, columns[i])
    print(classification_report(y_test.iloc[:,i], y_pred[:,i]))

Category 0 related
             precision    recall  f1-score   support

          0       0.66      0.35      0.46      2047
          1       0.82      0.94      0.88      6604

avg / total       0.79      0.80      0.78      8651

Category 1 request
             precision    recall  f1-score   support

          0       0.91      0.97      0.94      7137
          1       0.77      0.54      0.64      1514

avg / total       0.88      0.89      0.88      8651

Category 2 offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      8616
          1       0.22      0.06      0.09        35

avg / total       0.99      1.00      0.99      8651

Category 3 aid_related
             precision    recall  f1-score   support

          0       0.75      0.87      0.80      5059
          1       0.76      0.58      0.66      3592

avg / total       0.75      0.75      0.74      8651

Category 4 medical_help
             precision    recall  f1-s

Choosing the second attempt as the model to use.

### 9. Export your model as a pickle file

In [28]:
# picking 
pickle.dump(pipeline_3, open('classifier.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.